# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Store the CSV in WeatherPy into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yerbogachen,61.2767,108.0108,16.79,96,73,3.83,RU,1635271106
1,1,nikolskoye,59.7035,30.7861,45.88,87,100,7.45,RU,1635270648
2,2,rikitea,-23.1203,-134.9692,75.11,79,100,18.75,PF,1635270625
3,3,hermanus,-34.4187,19.2345,64.80,82,94,18.16,ZA,1635270599
4,4,chiredzi,-21.0500,31.6667,73.44,60,35,8.50,ZW,1635271115


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#Configure GMAPS
gmaps.configure(api_key=g_key)

In [18]:
#heatmap of the humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig =gmaps.figure()

#add the heatmap layer to the figure
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
idealWeatherDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & \
    (city_data_df["Wind Speed"] > 10) & \
    (city_data_df["Cloudiness"] ==0)
    
].dropna()
#print(len(idealWeatherDF))

idealWeatherDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,port said,31.2565,32.2841,74.28,67,0,12.95,EG,1635271213
288,288,calama,-22.4667,-68.9333,72.09,13,0,25.32,CL,1635271821
416,416,comodoro rivadavia,-45.8667,-67.5000,75.06,23,0,33.38,AR,1635272144
532,532,matay,28.4190,30.7792,77.59,42,0,13.89,EG,1635272449
553,553,pimentel,-6.8367,-79.9342,70.09,64,0,16.11,PE,1635272502


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotelDF = idealWeatherDF[["City", "Country", "Lat", "Lng"]]
hotelDF["Hotel Name"] = ""
hotelDF

<ipython-input-20-fc802e03a402>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotelDF["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
45,port said,EG,31.2565,32.2841,
288,calama,CL,-22.4667,-68.9333,
416,comodoro rivadavia,AR,-45.8667,-67.5000,
532,matay,EG,28.4190,30.7792,
553,pimentel,PE,-6.8367,-79.9342,


In [23]:
#Set the parameters to search hotel

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Lopp through the dataframe usin iterrows()
for index, row in hotelDF.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    #use the lat and lng to se the locations in value s in the param dictionary
    params["location"] = f"{lat}, {lng}"
    
    #use the google apis places base url to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make request
    name_address =requests.get(base_url, params=params).json()
    
    #pprint(name_address) #print ["results"][0]["name"]
    
    try:
        hotelDF.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except:
        print(f"Error finding lodging in {row['City']}. Skipping......")
        pass

hotelDF

,City,Country,Lat,Lng,Hotel Name
45,port said,EG,31.2565,32.2841,Aracan Hotel
288,calama,CL,-22.4667,-68.9333,Diego De Almagro Calama
416,comodoro rivadavia,AR,-45.8667,-67.5000,Austral Hotel
532,matay,EG,28.4190,30.7792,Nag Aespen
553,pimentel,PE,-6.8367,-79.9342,Casona Del Muelle


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelDF.iterrows()]
locations = hotelDF[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))